In [22]:
# Imports
import pandas as pd
import plotly.express as px
from prettytable import PrettyTable
import plotly.graph_objects as go
import numpy as np
import random
import csv
import os
import json
import altair as alt
from tqdm import tqdm
import numpy as np
import seaborn as sns
#import dataframe_image as dfi
from numpy.random import seed
from numpy.random import randn
from statsmodels.graphics.gofplots import qqplot
from matplotlib import pyplot
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [23]:
WD = os.path.join("E:\\", "BUSTED_ModelTest")

#WD = "/Users/alex/Documents/BUSTEDS-MH"
DATA = os.path.join(WD, "tables", "Table_14-datasets_results_adjusted.csv")

In [24]:
print("Reading:", DATA)
df = pd.read_csv(DATA)
df = df.reset_index(drop=True)
df.index += 1
df

Reading: E:\BUSTED_ModelTest\tables\Table_14-datasets_results_adjusted.csv


,Gene,Method,Sequences,Codons,LRT p-value,Bonferroni p-value,Bonferroni significant,cAIC,CV(omega),CV(alpha),...,NUM_ER_SITES_OPTIMIZED_NULL,ER_SITES_OPTIMIZED_NULL,ER_SITES_OPTIMIZED_NULL_VALUES,NUM_ER_SITES_CONSTRAINED_loose,ER_SITES_CONSTRAINED_loose,NUM_ER_SITES_OPTIMIZED_NULL_loose,ER_SITES_OPTIMIZED_NULL_loose,DH_Rate,TH_Rate,TH_Rate_SI
1,COXI,BUSTED,21,510,5.000000e-01,0.0125,False,24384.439201,0.839072,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,COXI,BUSTED-MH,21,510,5.000000e-01,0.0125,False,24389.451070,0.814080,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,5.676599
3,COXI,BUSTEDS,21,510,5.000000e-01,0.0125,False,24288.101212,1.381566,1.330024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,COXI,BUSTEDS-MH,21,510,5.000000e-01,0.0125,False,24295.058286,0.701211,1.306716,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,4.371442
5,ENCenv,BUSTED,23,500,5.000000e-01,0.0125,False,13745.467295,1.203131,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,ENCenv,BUSTED-MH,23,500,5.000000e-01,0.0125,False,13749.208238,1.278712,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.086759,0.000000,0.000000
7,ENCenv,BUSTEDS,23,500,5.000000e-01,0.0125,False,13699.103102,1.254676,0.893186,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,ENCenv,BUSTEDS-MH,23,500,5.000000e-01,0.0125,False,13705.123029,1.282742,0.891511,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.012449,0.000000,0.000000
9,HIV_RT,BUSTED,476,335,0.000000e+00,0.0125,True,53749.794833,1.350796,NaN,...,16.0,35|48|64|75|104|122|151|162|163|174|181|188|20...,9.1341148092035|164.62539766982|30.45965877386...,84.0,6|11|20|32|35|36|39|40|41|43|48|49|60|64|65|69...,214.0,1|3|4|5|7|9|10|12|13|14|15|16|17|19|21|22|25|2...,NaN,NaN,NaN
10,HIV_RT,BUSTED-MH,476,335,0.000000e+00,0.0125,True,53757.215664,1.404597,NaN,...,12.0,48|64|75|122|151|162|174|181|188|215|228|245,19.05033527880322|8.958711796619502|6.01662545...,85.0,6|11|20|32|35|36|39|40|41|43|48|49|60|64|65|69...,199.0,1|3|4|5|7|9|10|12|13|14|15|16|19|21|25|26|27|2...,0.000000,0.000000,0.000000


In [25]:
df.columns

Index(['Gene', 'Method', 'Sequences', 'Codons', 'LRT p-value',
       'Bonferroni p-value', 'Bonferroni significant', 'cAIC', 'CV(omega)',
       'CV(alpha)', 'w1', 'p1', 'w2', 'p2', 'w3', 'p3', 'SRV1', 'SRV_p1',
       'SRV2', 'SRV_p2', 'SRV3', 'SRV_p3', 'NUM_ER_SITES_CONSTRAINED',
       'ER_SITES_CONSTRAINED', 'NUM_ER_SITES_OPTIMIZED_NULL',
       'ER_SITES_OPTIMIZED_NULL', 'ER_SITES_OPTIMIZED_NULL_VALUES',
       'NUM_ER_SITES_CONSTRAINED_loose', 'ER_SITES_CONSTRAINED_loose',
       'NUM_ER_SITES_OPTIMIZED_NULL_loose', 'ER_SITES_OPTIMIZED_NULL_loose',
       'DH_Rate', 'TH_Rate', 'TH_Rate_SI'],
      dtype='object')

## Process

In [26]:
#for index, row in tqdm(df.iterrows()):
#    print(index)


# Set of genes

gene_set = set(df["Gene"])

models = ["BUSTED", "BUSTEDS", "BUSTED-MH", "BUSTEDS-MH"]

counts_dict = {}

for model in models:
    counts_dict[model] = {"count": 0, 
                          "countStrongPreference": 0, # delta cAIC of more than 5.
                          "genes":[],
                         "genesStrongPreference":[],
                         "median": 0}

dfh_holder = []

for gene in gene_set:
    dfh = df[df["Gene"] == gene] # df holder for each gene
    #dfh["ΔcAIC"] = ""
    cAIC_values = []
    
    for method in models:
        cAIC_val = float(dfh[dfh["Method"] == method]["cAIC"])
        cAIC_values.append(cAIC_val)
    #end for
    
    best_model_cAIC = min(cAIC_values)
    
    # Which method does this correspond to?
    best_model = ""
    
    strong_preference = False
    
    for method in models:
        cAIC_val = float(dfh[dfh["Method"] == method]["cAIC"])
        if cAIC_val == best_model_cAIC:
            best_model = method
        else:
            delta_cAIC = cAIC_val - best_model_cAIC
            if delta_cAIC > 5:
                strong_preference = True
            #end if
        #end if
    #end for
    
    print("# Gene:", gene, "Best model cAIC:", best_model_cAIC, best_model)
    
    # counts
    counts_dict[best_model]["count"] += 1
    counts_dict[best_model]["genes"].append(gene)
    
    if strong_preference == True:
        counts_dict[best_model]["countStrongPreference"] += 1
        counts_dict[best_model]["genesStrongPreference"].append(gene)
    
    #delta cAIC
    #for method in models:
    #    delta_cAIC = 0
    #    cAIC_val = float(dfh[dfh["Method"] == method]["cAIC"])
    #    delta_cAIC = cAIC_val - best_model_cAIC
    #    print("Method:", method, "Delta cAIC:", delta_cAIC)
    #    dfh[dfh["Method"] == method]["ΔcAIC"] = delta_cAIC
    
    for index, row in dfh.iterrows():
        dfh_gene = row["Gene"]
        #dfh_gene = row["Gene"]
        cAIC_val = row["cAIC"]
        delta_cAIC = cAIC_val - best_model_cAIC
        dfh.at[index, "ΔcAIC"] = delta_cAIC

    # Strong model preference
    
    
        
    dfh_holder.append(dfh)
#end for
    
    

# Gene: adh Best model cAIC: 9354.009509023232 BUSTEDS
# Gene: InfluenzaA Best model cAIC: 23230.19555299281 BUSTEDS-MH
# Gene: HIV_RT Best model cAIC: 52026.76925371627 BUSTEDS
# Gene: VOI_SPIKE Best model cAIC: 17807.6495224851 BUSTEDS
# Gene: ENCenv Best model cAIC: 13699.10310242903 BUSTEDS
# Gene: camelid Best model cAIC: 33668.70141504951 BUSTEDS-MH
# Gene: bglobin Best model cAIC: 7384.827434932465 BUSTEDS-MH
# Gene: lysozyme Best model cAIC: 2145.579372119591 BUSTEDS
# Gene: flavNS5 Best model cAIC: 18488.57869611563 BUSTEDS-MH
# Gene: yokoyama.rh1.cds.mod.1-990 Best model cAIC: 25895.72154682217 BUSTEDS-MH
# Gene: HIVvif Best model cAIC: 6913.086831262467 BUSTEDS-MH
# Gene: lysin Best model cAIC: 8761.171229912901 BUSTEDS
# Gene: HepatitisD Best model cAIC: 10423.69306414134 BUSTEDS-MH
# Gene: COXI Best model cAIC: 24288.10121234685 BUSTEDS


c:\python39\lib\site-packages\pandas\core\indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
c:\python39\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [27]:
result = pd.concat(dfh_holder)
result

,Gene,Method,Sequences,Codons,LRT p-value,Bonferroni p-value,Bonferroni significant,cAIC,CV(omega),CV(alpha),...,ER_SITES_OPTIMIZED_NULL,ER_SITES_OPTIMIZED_NULL_VALUES,NUM_ER_SITES_CONSTRAINED_loose,ER_SITES_CONSTRAINED_loose,NUM_ER_SITES_OPTIMIZED_NULL_loose,ER_SITES_OPTIMIZED_NULL_loose,DH_Rate,TH_Rate,TH_Rate_SI,ΔcAIC
29,adh,BUSTED,23,254,2.624235e-04,0.0125,True,9371.692413,1.085644,NaN,...,163|170,11.3761195693454|8.965332511952935,63.0,2|3|4|6|8|9|23|26|29|35|39|40|42|44|46|49|57|6...,152.0,1|4|6|10|12|14|15|16|17|19|20|22|24|25|26|27|3...,NaN,NaN,NaN,17.682904
30,adh,BUSTED-MH,23,254,2.107436e-02,0.0125,False,9377.068424,1.394360,NaN,...,NaN,NaN,64.0,2|3|4|6|8|9|23|26|29|35|39|40|42|44|46|49|51|5...,124.0,4|6|8|9|10|12|15|16|18|19|20|22|25|26|32|34|35...,0.087721,0.000000,0.000000,23.058915
31,adh,BUSTEDS,23,254,1.815529e-03,0.0125,True,9354.009509,1.383019,0.574009,...,170,6.83283045298949,60.0,2|3|4|5|6|8|9|23|26|35|39|40|44|46|49|51|57|68...,122.0,1|4|6|7|8|10|12|15|16|19|22|23|26|29|30|32|34|...,NaN,NaN,NaN,0.000000
32,adh,BUSTEDS-MH,23,254,1.948912e-02,0.0125,False,9359.111669,1.383861,0.552316,...,NaN,NaN,60.0,2|3|4|5|6|8|9|23|26|35|39|40|44|46|49|51|57|68...,114.0,4|6|8|9|12|15|16|19|22|23|26|29|32|34|35|36|37...,0.000000,0.033096,3.364916,5.102160
21,InfluenzaA,BUSTED,349,329,6.970205e-09,0.0125,True,23863.727926,1.330142,NaN,...,133|135|157|159|213,13.46894629398|9.870295927657809|59.6784205482...,91.0,1|2|3|15|31|32|44|45|46|47|48|50|53|57|59|62|7...,189.0,3|4|7|8|9|11|13|14|15|18|19|20|21|24|25|26|27|...,NaN,NaN,NaN,633.532373
22,InfluenzaA,BUSTED-MH,349,329,4.801551e-01,0.0125,False,23861.491457,0.923508,NaN,...,NaN,NaN,93.0,1|2|3|22|31|32|44|45|46|47|48|50|53|57|59|62|7...,179.0,3|7|9|12|15|16|17|18|20|21|24|26|28|30|33|35|4...,0.067817,0.014964,0.000000,631.295904
23,InfluenzaA,BUSTEDS,349,329,1.063992e-01,0.0125,False,23242.618200,0.837749,0.870636,...,NaN,NaN,122.0,1|2|3|5|11|15|19|21|31|34|36|38|44|45|46|47|48...,133.0,1|2|3|5|6|7|8|11|15|19|21|23|25|26|27|28|29|31...,NaN,NaN,NaN,12.422647
24,InfluenzaA,BUSTEDS-MH,349,329,5.000000e-01,0.0125,False,23230.195553,0.474401,0.877723,...,NaN,NaN,NaN,NaN,NaN,NaN,0.062803,0.014882,0.000000,0.000000
9,HIV_RT,BUSTED,476,335,0.000000e+00,0.0125,True,53749.794833,1.350796,NaN,...,35|48|64|75|104|122|151|162|163|174|181|188|20...,9.1341148092035|164.62539766982|30.45965877386...,84.0,6|11|20|32|35|36|39|40|41|43|48|49|60|64|65|69...,214.0,1|3|4|5|7|9|10|12|13|14|15|16|17|19|21|22|25|2...,NaN,NaN,NaN,1723.025579
10,HIV_RT,BUSTED-MH,476,335,0.000000e+00,0.0125,True,53757.215664,1.404597,NaN,...,48|64|75|122|151|162|174|181|188|215|228|245,19.05033527880322|8.958711796619502|6.01662545...,85.0,6|11|20|32|35|36|39|40|41|43|48|49|60|64|65|69...,199.0,1|3|4|5|7|9|10|12|13|14|15|16|19|21|25|26|27|2...,0.000000,0.000000,0.000000,1730.446410


In [50]:
result.columns

Index(['Gene', 'Method', 'Sequences', 'Codons', 'LRT p-value',
       'Bonferroni p-value', 'Bonferroni significant', 'cAIC', 'CV(omega)',
       'CV(alpha)', 'w1', 'p1', 'w2', 'p2', 'w3', 'p3', 'SRV1', 'SRV_p1',
       'SRV2', 'SRV_p2', 'SRV3', 'SRV_p3', 'NUM_ER_SITES_CONSTRAINED',
       'ER_SITES_CONSTRAINED', 'NUM_ER_SITES_OPTIMIZED_NULL',
       'ER_SITES_OPTIMIZED_NULL', 'ER_SITES_OPTIMIZED_NULL_VALUES',
       'NUM_ER_SITES_CONSTRAINED_loose', 'ER_SITES_CONSTRAINED_loose',
       'NUM_ER_SITES_OPTIMIZED_NULL_loose', 'ER_SITES_OPTIMIZED_NULL_loose',
       'DH_Rate', 'TH_Rate', 'TH_Rate_SI', 'ΔcAIC'],
      dtype='object')

In [28]:
delta_cAIC = result[result["ΔcAIC"] > 0]
BUSTEDS_preferred = delta_cAIC[delta_cAIC["Method"] == "BUSTEDS"]
BUSTEDSMH_preferred = delta_cAIC[delta_cAIC["Method"] == "BUSTEDS-MH"]

print("# Number of genes preferring BUSTEDS (by any amount, cAIC):", BUSTEDS_preferred.shape[0])
print("# Number of genes preferring BUSTEDS-MH (by any amount, cAIC):", BUSTEDSMH_preferred.shape[0])

y = BUSTEDSMH_preferred["ΔcAIC"].median()
print("# Median ΔcAIC genes preferring BUSTEDS-MH:", round(y, 1))

Q3 = np.quantile(BUSTEDSMH_preferred["ΔcAIC"], 0.75)
Q1 = np.quantile(BUSTEDSMH_preferred["ΔcAIC"], 0.25)
IQR = Q3 - Q1
print("# IQR (ΔcAIC):", round(IQR, 1))

z = BUSTEDS_preferred["ΔcAIC"].median()
print("# Median ΔcAIC genes preferring BUSTEDS:", round(z, 1))

Q3 = np.quantile(BUSTEDS_preferred["ΔcAIC"], 0.75)
Q1 = np.quantile(BUSTEDS_preferred["ΔcAIC"], 0.25)
IQR = Q3 - Q1
print("# IQR (ΔcAIC):", round(IQR, 1))

# ER Analysis
a = BUSTEDSMH_preferred["NUM_ER_SITES_OPTIMIZED_NULL"].median()
a_1 = BUSTEDSMH_preferred["NUM_ER_SITES_OPTIMIZED_NULL"].min()
a_2 = BUSTEDSMH_preferred["NUM_ER_SITES_OPTIMIZED_NULL"].max()
b = BUSTEDS_preferred["NUM_ER_SITES_OPTIMIZED_NULL"].median()
b_1 = BUSTEDS_preferred["NUM_ER_SITES_OPTIMIZED_NULL"].min()
b_2 = BUSTEDS_preferred["NUM_ER_SITES_OPTIMIZED_NULL"].max()

print("# Median ER Sites genes preferring BUSTEDS-MH:", round(a, 1))
print("# Minimum ER Sites genes preferring BUSTEDS-MH:", round(a_1, 1))
print("# Minimum ER Sites genes preferring BUSTEDS-MH:", round(a_2, 1))

Q3 = np.quantile(BUSTEDSMH_preferred["NUM_ER_SITES_OPTIMIZED_NULL"], 0.75)
Q1 = np.quantile(BUSTEDSMH_preferred["NUM_ER_SITES_OPTIMIZED_NULL"], 0.25)
IQR = Q3 - Q1
print("# IQR (ΔcAIC):", round(IQR, 1))

print("# Median ER Sites genes preferring BUSTEDS:", round(b, 1))
print("# Minimum ER Sites genes preferring BUSTEDS-MH:", round(b_1, 1))
print("# Minimum ER Sites genes preferring BUSTEDS-MH:", round(b_2, 1))

Q3 = np.quantile(BUSTEDS_preferred["NUM_ER_SITES_OPTIMIZED_NULL"], 0.75)
Q1 = np.quantile(BUSTEDS_preferred["NUM_ER_SITES_OPTIMIZED_NULL"], 0.25)
IQR = Q3 - Q1
print("# IQR (ΔcAIC):", round(IQR, 1))


print()

a = BUSTEDSMH_preferred["NUM_ER_SITES_OPTIMIZED_NULL_loose"].median()
a_1 = BUSTEDSMH_preferred["NUM_ER_SITES_OPTIMIZED_NULL_loose"].min()
a_2 = BUSTEDSMH_preferred["NUM_ER_SITES_OPTIMIZED_NULL_loose"].max()
b = BUSTEDS_preferred["NUM_ER_SITES_OPTIMIZED_NULL_loose"].median()
b_1 = BUSTEDS_preferred["NUM_ER_SITES_OPTIMIZED_NULL_loose"].min()
b_2 = BUSTEDS_preferred["NUM_ER_SITES_OPTIMIZED_NULL_loose"].max()

print("# Median ER Sites genes preferring BUSTEDS-MH _loose:", round(a, 1))
print("# Minimum ER Sites genes preferring BUSTEDS-MH _loose:", round(a_1, 1))
print("# Maximum ER Sites genes preferring BUSTEDS-MH _loose:", round(a_2, 1))

Q3 = np.quantile(BUSTEDSMH_preferred["NUM_ER_SITES_OPTIMIZED_NULL_loose"], 0.75)
Q1 = np.quantile(BUSTEDSMH_preferred["NUM_ER_SITES_OPTIMIZED_NULL_loose"], 0.25)
IQR = Q3 - Q1
print("# IQR (ΔcAIC) _loose:", round(IQR, 1))

print("# Median ER Sites genes preferring BUSTEDS _loose:", round(b, 1))
print("# Minimum ER Sites genes preferring BUSTEDS _loose:", round(b_1, 1))
print("# Maximum ER Sites genes preferring BUSTEDS _loose:", round(b_2, 1))

Q3 = np.quantile(BUSTEDS_preferred["NUM_ER_SITES_OPTIMIZED_NULL_loose"], 0.75)
Q1 = np.quantile(BUSTEDS_preferred["NUM_ER_SITES_OPTIMIZED_NULL_loose"], 0.25)
IQR = Q3 - Q1
print("# IQR (ΔcAIC)_loose:", round(IQR, 1))

# Number of genes preferring BUSTEDS (by any amount, cAIC): 7
# Number of genes preferring BUSTEDS-MH (by any amount, cAIC): 7
# Median ΔcAIC genes preferring BUSTEDS-MH: 6.3
# IQR (ΔcAIC): 3.1
# Median ΔcAIC genes preferring BUSTEDS: 25.4
# IQR (ΔcAIC): 22.7
# Median ER Sites genes preferring BUSTEDS-MH: 1.5
# Minimum ER Sites genes preferring BUSTEDS-MH: 0.0
# Minimum ER Sites genes preferring BUSTEDS-MH: 3.0
# IQR (ΔcAIC): nan
# Median ER Sites genes preferring BUSTEDS: 5.0
# Minimum ER Sites genes preferring BUSTEDS-MH: 0.0
# Minimum ER Sites genes preferring BUSTEDS-MH: 28.0
# IQR (ΔcAIC): 7.0

# Median ER Sites genes preferring BUSTEDS-MH _loose: 142.5
# Minimum ER Sites genes preferring BUSTEDS-MH _loose: 62.0
# Maximum ER Sites genes preferring BUSTEDS-MH _loose: 579.0
# IQR (ΔcAIC) _loose: nan
# Median ER Sites genes preferring BUSTEDS _loose: 127.0
# Minimum ER Sites genes preferring BUSTEDS _loose: 57.0
# Maximum ER Sites genes preferring BUSTEDS _loose: 201.0
# IQR (ΔcAIC)_

## ER Analysis

In [29]:
import altair as alt
from vega_datasets import data

# Since the data is more than 5,000 rows we'll import it from a URL
source = data.seattle_temps.url

alt.Chart(
    source,
    title="2010 Daily High Temperature (F) in Seattle, WA"
).mark_rect().encode(
    x='date(date):O',
    y='month(date):O',
    color=alt.Color('max(temp):Q', scale=alt.Scale(scheme="inferno")),
    tooltip=[
        alt.Tooltip('monthdate(date):T', title='Date'),
        alt.Tooltip('max(temp):Q', title='Max Temp')
    ]
).properties(width=550)

alt.Chart(...)

In [30]:
# ER_SITES_OPTIMIZED_NULL
import altair as alt
from vega_datasets import data

# Since the data is more than 5,000 rows we'll import it from a URL
source = data.seattle_temps.url

alt.Chart(
    source,
    title="2010 Daily High Temperature (F) in Seattle, WA"
).mark_rect().encode(
    x='date(date):O',
    y='month(date):O',
    color=alt.Color('max(temp):Q', scale=alt.Scale(scheme="inferno")),
).properties(width=550)



alt.Chart(...)

In [51]:
result_heatmap = result[["Gene", "Method", "ER_SITES_OPTIMIZED_NULL", "ER_SITES_OPTIMIZED_NULL_VALUES"]]
result_heatmap 



,Gene,Method,ER_SITES_OPTIMIZED_NULL,ER_SITES_OPTIMIZED_NULL_VALUES
29,adh,BUSTED,163|170,11.3761195693454|8.965332511952935
30,adh,BUSTED-MH,NaN,NaN
31,adh,BUSTEDS,170,6.83283045298949
32,adh,BUSTEDS-MH,NaN,NaN
21,InfluenzaA,BUSTED,133|135|157|159|213,13.46894629398|9.870295927657809|59.6784205482...
22,InfluenzaA,BUSTED-MH,NaN,NaN
23,InfluenzaA,BUSTEDS,NaN,NaN
24,InfluenzaA,BUSTEDS-MH,NaN,NaN
9,HIV_RT,BUSTED,35|48|64|75|104|122|151|162|163|174|181|188|20...,9.1341148092035|164.62539766982|30.45965877386...
10,HIV_RT,BUSTED-MH,48|64|75|122|151|162|174|181|188|215|228|245,19.05033527880322|8.958711796619502|6.01662545...


In [52]:
data_dict = {}

count = 0
for index, row in result_heatmap.iterrows():
    gene     = row["Gene"]
    method   = row["Method"]
    er_sites = row["ER_SITES_OPTIMIZED_NULL"]
    
    if type(er_sites) == float:
        continue
    #end if
    
    #data_dict[count] = {"Gene": gene, "Method": method}
    print(gene, method, er_sites.split("|"))
    #print("looping")
    for n, item in enumerate(er_sites.split("|")):
        #print(item, count)
        #data_dict[count].update({"ER": item})
        data_dict[count] = {"ER_SITE": item, 
                            "Gene": gene, 
                            "Method": method, 
                            "ER_VALUE": row["ER_SITES_OPTIMIZED_NULL_VALUES"].split("|")[n]}
        count += 1
    #end for
    

adh BUSTED ['163', '170']
adh BUSTEDS ['170']
InfluenzaA BUSTED ['133', '135', '157', '159', '213']
HIV_RT BUSTED ['35', '48', '64', '75', '104', '122', '151', '162', '163', '174', '181', '188', '207', '215', '228', '245']
HIV_RT BUSTED-MH ['48', '64', '75', '122', '151', '162', '174', '181', '188', '215', '228', '245']
HIV_RT BUSTEDS ['48', '64', '135', '151', '163', '178', '188', '200', '215', '228']
HIV_RT BUSTEDS-MH ['188', '215', '228']
VOI_SPIKE BUSTED ['157', '212', '220', '221', '253', '378', '497']
VOI_SPIKE BUSTEDS ['378', '497']
camelid BUSTED ['4', '5', '8', '9', '10', '12', '17', '18', '20', '21', '23', '24', '25', '26', '28', '29', '30', '32', '33', '35', '38', '39', '40', '41', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '64', '66', '67', '68', '71', '72', '73', '75', '78', '80', '82', '86', '92']
camelid BUSTED-MH ['11', '25', '29', '32', '33', '41', '50', '51', '52', '54', '55', '56', '57', '67', '78']
camelid BUSTEDS ['5', '10', '23', '24', '25', '26',

In [53]:
data_dict

{0: {'ER_SITE': '163',
  'Gene': 'adh',
  'Method': 'BUSTED',
  'ER_VALUE': '11.3761195693454'},
 1: {'ER_SITE': '170',
  'Gene': 'adh',
  'Method': 'BUSTED',
  'ER_VALUE': '8.965332511952935'},
 2: {'ER_SITE': '170',
  'Gene': 'adh',
  'Method': 'BUSTEDS',
  'ER_VALUE': '6.83283045298949'},
 3: {'ER_SITE': '133',
  'Gene': 'InfluenzaA',
  'Method': 'BUSTED',
  'ER_VALUE': '13.46894629398'},
 4: {'ER_SITE': '135',
  'Gene': 'InfluenzaA',
  'Method': 'BUSTED',
  'ER_VALUE': '9.870295927657809'},
 5: {'ER_SITE': '157',
  'Gene': 'InfluenzaA',
  'Method': 'BUSTED',
  'ER_VALUE': '59.67842054827001'},
 6: {'ER_SITE': '159',
  'Gene': 'InfluenzaA',
  'Method': 'BUSTED',
  'ER_VALUE': '222.1236343064687'},
 7: {'ER_SITE': '213',
  'Gene': 'InfluenzaA',
  'Method': 'BUSTED',
  'ER_VALUE': '5.672080431621395'},
 8: {'ER_SITE': '35',
  'Gene': 'HIV_RT',
  'Method': 'BUSTED',
  'ER_VALUE': '9.1341148092035'},
 9: {'ER_SITE': '48',
  'Gene': 'HIV_RT',
  'Method': 'BUSTED',
  'ER_VALUE': '164.6253

In [54]:
df = pd.DataFrame.from_dict(data_dict, orient='index')
df

,ER_SITE,Gene,Method,ER_VALUE
0,163,adh,BUSTED,11.3761195693454
1,170,adh,BUSTED,8.965332511952935
2,170,adh,BUSTEDS,6.83283045298949
3,133,InfluenzaA,BUSTED,13.46894629398
4,135,InfluenzaA,BUSTED,9.870295927657809
...,...,...,...,...
278,90,HepatitisD,BUSTEDS,15.79110615207451
279,117,HepatitisD,BUSTEDS,6.460792777385842
280,140,HepatitisD,BUSTEDS,7.518987982335722
281,145,HepatitisD,BUSTEDS,8.151648268018537


In [55]:
df[df["Gene"] == "HepatitisD"]

,ER_SITE,Gene,Method,ER_VALUE
259,6,HepatitisD,BUSTED,24.11463517158267
260,9,HepatitisD,BUSTED,25.31626756704223
261,13,HepatitisD,BUSTED,70.086472822797
262,23,HepatitisD,BUSTED,11.91818019529393
263,24,HepatitisD,BUSTED,14.53540377778277
264,28,HepatitisD,BUSTED,42.92960645248242
265,31,HepatitisD,BUSTED,5.169131569576953
266,35,HepatitisD,BUSTED,7.725551773552785
267,90,HepatitisD,BUSTED,20.10538356389257
268,117,HepatitisD,BUSTED,25.75394921432199


In [56]:
alt.Chart(
    df[df["Gene"] == "HepatitisD"].sort_values(by=['ER_SITE']),
    title="2010 Daily High Temperature (F) in Seattle, WA"
).mark_rect().encode(
    x=alt.X('ER_SITE:O', sort=['ER']),
    y='Method'
).properties(width=550)

alt.Chart(...)

In [57]:


gene_tag = "HIV_RT"
sites = df[df["Gene"] == gene_tag]["ER_SITE"].to_list()
print(sites)
sites = [int(x) for x in sites]
sites.sort(reverse = False)
print(sites)
sites = [str(x) for x in sites]
print(sites)

alt.Chart(
    df[df["Gene"] == gene_tag],
    title="Evidence ratio sites for HIV reverse transcriptase"
).mark_rect().encode(
    x=alt.X('ER_SITE:O', sort=sites),
    y='Method'
).properties(width=550, height=200)

['35', '48', '64', '75', '104', '122', '151', '162', '163', '174', '181', '188', '207', '215', '228', '245', '48', '64', '75', '122', '151', '162', '174', '181', '188', '215', '228', '245', '48', '64', '135', '151', '163', '178', '188', '200', '215', '228', '188', '215', '228']
[35, 48, 48, 48, 64, 64, 64, 75, 75, 104, 122, 122, 135, 151, 151, 151, 162, 162, 163, 163, 174, 174, 178, 181, 181, 188, 188, 188, 188, 200, 207, 215, 215, 215, 215, 228, 228, 228, 228, 245, 245]
['35', '48', '48', '48', '64', '64', '64', '75', '75', '104', '122', '122', '135', '151', '151', '151', '162', '162', '163', '163', '174', '174', '178', '181', '181', '188', '188', '188', '188', '200', '207', '215', '215', '215', '215', '228', '228', '228', '228', '245', '245']


alt.Chart(...)

In [69]:


gene_tag = "HIV_RT"
sites = df[df["Gene"] == gene_tag]["ER_SITE"].to_list()
print(sites)
sites = [int(x) for x in sites]
sites.sort(reverse = False)
print(sites)
sites = [str(x) for x in sites]
print(sites)

alt.Chart(
    df[df["Gene"] == gene_tag],
    title="Evidence ratio sites for HIV reverse transcriptase"
).mark_rect().encode(
    x=alt.X('ER_SITE:O', sort=sites),
    y='Method',
    color = "Method"
).properties(width=550, height=200)

['35', '48', '64', '75', '104', '122', '151', '162', '163', '174', '181', '188', '207', '215', '228', '245', '48', '64', '75', '122', '151', '162', '174', '181', '188', '215', '228', '245', '48', '64', '135', '151', '163', '178', '188', '200', '215', '228', '188', '215', '228']
[35, 48, 48, 48, 64, 64, 64, 75, 75, 104, 122, 122, 135, 151, 151, 151, 162, 162, 163, 163, 174, 174, 178, 181, 181, 188, 188, 188, 188, 200, 207, 215, 215, 215, 215, 228, 228, 228, 228, 245, 245]
['35', '48', '48', '48', '64', '64', '64', '75', '75', '104', '122', '122', '135', '151', '151', '151', '162', '162', '163', '163', '174', '174', '178', '181', '181', '188', '188', '188', '188', '200', '207', '215', '215', '215', '215', '228', '228', '228', '228', '245', '245']


alt.Chart(...)

In [66]:


gene_tag = "HIV_RT"
sites = df[df["Gene"] == gene_tag]["ER_SITE"].to_list()
print(sites)
sites = [int(x) for x in sites]
sites.sort(reverse = False)
print(sites)
sites = [str(x) for x in sites]
print(sites)

alt.Chart(
    df[df["Gene"] == gene_tag],
    title="Evidence ratio sites for HIV reverse transcriptase"
).mark_circle().encode(
    x=alt.X('ER_SITE:O', sort=sites),
    y='Method',
    size = "ER_VALUE",
    color = "Method"
).properties(width=550, height=200)

['35', '48', '64', '75', '104', '122', '151', '162', '163', '174', '181', '188', '207', '215', '228', '245', '48', '64', '75', '122', '151', '162', '174', '181', '188', '215', '228', '245', '48', '64', '135', '151', '163', '178', '188', '200', '215', '228', '188', '215', '228']
[35, 48, 48, 48, 64, 64, 64, 75, 75, 104, 122, 122, 135, 151, 151, 151, 162, 162, 163, 163, 174, 174, 178, 181, 181, 188, 188, 188, 188, 200, 207, 215, 215, 215, 215, 228, 228, 228, 228, 245, 245]
['35', '48', '48', '48', '64', '64', '64', '75', '75', '104', '122', '122', '135', '151', '151', '151', '162', '162', '163', '163', '174', '174', '178', '181', '181', '188', '188', '188', '188', '200', '207', '215', '215', '215', '215', '228', '228', '228', '228', '245', '245']


alt.Chart(...)

In [59]:
df

,ER_SITE,Gene,Method,ER_VALUE
0,163,adh,BUSTED,11.3761195693454
1,170,adh,BUSTED,8.965332511952935
2,170,adh,BUSTEDS,6.83283045298949
3,133,InfluenzaA,BUSTED,13.46894629398
4,135,InfluenzaA,BUSTED,9.870295927657809
...,...,...,...,...
278,90,HepatitisD,BUSTEDS,15.79110615207451
279,117,HepatitisD,BUSTEDS,6.460792777385842
280,140,HepatitisD,BUSTEDS,7.518987982335722
281,145,HepatitisD,BUSTEDS,8.151648268018537


In [60]:
from altair_saver import save

In [61]:
for gene in set(df["Gene"]):

    gene_tag = gene
    sites = df[df["Gene"] == gene_tag]["ER"].to_list()
    #print(sites)
    sites = [int(x) for x in sites]
    sites.sort(reverse = False)
    #print(sites)
    sites = [str(x) for x in sites]
    #print(sites)

    chart = alt.Chart(
        df[df["Gene"] == gene_tag],
        title="Evidence ratio sites for " + gene + " transcriptase"
    ).mark_rect().encode(
        x=alt.X('ER:O', sort=sites),
        y='Method'
    ).properties(width=550, height=200)
    
    save(chart, "ER_HEATMAP_" + gene + ".png")

KeyError: 'ER'

In [ ]:
# NUM_ER_SITES_OPTIMIZED_NULL



In [ ]:
result.columns

In [ ]:
counts_dict

In [ ]:
dfh

## End of file

In [ ]:
#for index, row in tqdm(df.iterrows()):
#    print(index)


# Set of genes

gene_set = set(df["Gene"])

models = ["BUSTED", "BUSTEDS", "BUSTED-MH", "BUSTEDS-MH"]

counts_dict = {}

for model in models:
    counts_dict[model] = 0
    
#counts_dict = {"BUSTED": 0,
#              "BUSTEDS": 0,
#              "BUSTEDS-MH": 0,
#              "BUSTED-MH": 0}

dfh_holder = []

models = []

for gene in gene_set:
    dfh = df[df["Gene"] == gene] # df holder for each gene
    
    cAIC_values = []
    
    for method in models:
        cAIC_val = float(dfh[dfh["Method"] == method]["cAIC"])
        cAIC_values.append(cAIC_val)
        counts_dict[method].update({"Genes": })
    #BUSTED_cAIC     = float(dfh[dfh["Method"] == "BUSTED"]["cAIC"])    
    #BUSTEDS_cAIC    = float(dfh[dfh["Method"] == "BUSTEDS"]["cAIC"])    
    #BUSTED_MH_cAIC  = float(dfh[dfh["Method"] == "BUSTED-MH"]["cAIC"])    
    #BUSTEDS_MH_cAIC = float(dfh[dfh["Method"] == "BUSTEDS-MH"]["cAIC"])    
    
    #best_model_cAIC = min(BUSTED_cAIC, BUSTEDS_cAIC, BUSTED_MH_cAIC ,BUSTEDS_MH_cAIC)
    
    best_model_cAIC = min(cAIC_values)
    
    # Which method does this correspond to?
    best_model = ""
    
    if BUSTED_cAIC == best_model_cAIC:
        best_model = "BUSTED"
    elif BUSTEDS_cAIC == best_model_cAIC:
        best_model = "BUSTEDS"
    elif BUSTED_MH_cAIC == best_model_cAIC:
        best_model = "BUSTED-MH"
    elif BUSTEDS_MH_cAIC == best_model_cAIC:
        best_model = "BUSTEDS-MH"
    else:
        pass
    #end if
    
    print("# Gene:", gene, "Best model cAIC:", best_model_cAIC, best_model)
    
    # counts
    counts_dict[best_model]["count"] += 1
    
    #delta cAIC
    
    for cAIC_value in 
    
    
    

In [ ]:
#for index, row in tqdm(df.iterrows()):
#    print(index)


# Set of genes

gene_set = set(df["Gene"])

models = ["BUSTED", "BUSTEDS", "BUSTED-MH", "BUSTEDS-MH"]

counts_dict = {}

for model in models:
    counts_dict[model] = {"count": 0, "genes":[]}
    
#counts_dict = {"BUSTED": 0,
#              "BUSTEDS": 0,
#              "BUSTEDS-MH": 0,
#              "BUSTED-MH": 0}

dfh_holder = []

models = []

for gene in gene_set:
    dfh = df[df["Gene"] == gene] # df holder for each gene
    
    BUSTED_cAIC     = float(dfh[dfh["Method"] == "BUSTED"]["cAIC"])    
    BUSTEDS_cAIC    = float(dfh[dfh["Method"] == "BUSTEDS"]["cAIC"])    
    BUSTED_MH_cAIC  = float(dfh[dfh["Method"] == "BUSTED-MH"]["cAIC"])    
    BUSTEDS_MH_cAIC = float(dfh[dfh["Method"] == "BUSTEDS-MH"]["cAIC"])    
    
    best_model_cAIC = min(BUSTED_cAIC, BUSTEDS_cAIC, BUSTED_MH_cAIC ,BUSTEDS_MH_cAIC)
    
    # Which method does this correspond to?
    best_model = ""
    
    if BUSTED_cAIC == best_model_cAIC:
        best_model = "BUSTED"
    elif BUSTEDS_cAIC == best_model_cAIC:
        best_model = "BUSTEDS"
    elif BUSTED_MH_cAIC == best_model_cAIC:
        best_model = "BUSTED-MH"
    elif BUSTEDS_MH_cAIC == best_model_cAIC:
        best_model = "BUSTEDS-MH"
    else:
        pass
    #end if
    
    print("# Gene:", gene, "Best model cAIC:", best_model_cAIC, best_model)
    
    # counts
    counts_dict[best_model]["count"] += 1
    counts_dict[best_model]["genes"].append(gene)
    
    #delta cAIC
    
    
    
    